# Limpieza de datos. Apply. Expresiones regulares.

La limpieza es un paso necesario en todo proyecto de datos. 

Podemos resumir el proceso de limpieza de datos en las siguientes cinco tareas:

**1. Resolver problemas de formato y asignar tipos de datos correctos.**

Por ejemplo, cuando al pasar de CSV a Pandas una fecha no se importa correctamente como puede ser el caso de un campo fecha donde se importa 20090609231247 en lugar de 2009-06-09 23:12:47.

El formato en que se encuentran los datos determina qué operaciones pueden realizarse sobre ellos.

**2. Estandarizar categorías.**

Cuando los datos se recolectaron con un sisstema que no tiene valores tipificadaos, valores que representan la misma categoría pueden estar expresados de forma distinta. Por ejemplo: Arg, AR, Argentina

**3. Corregir valores erróneos.**

Por ejemplo: un valor numérico o inválido para describir el género; o una edad representada por un número negativo o mucho mayor que 100.

**4. Completar datos faltantes.**

Los datasets del mundo real suelen venir con datos faltantes que responden a información que se perdió o nunca se recolectó. Existen varias técnicas para completar datos faltantes. Al proceso de completar datos faltantes se lo llama "imputación".

**5. Organizar el dataset.**

Es importante estructurar las filas y columnas de la forma más conveniente. Para hacerlo se pueden aplicar las reglas del "tidy data".



## Dataset

El Met (Metropolitan Museum of Art) provee datasets de información de más de 420.000 piezas de arte en su colección.

https://github.com/metmuseum/openaccess/

Los problemas que presentan este dataset incluyen:
* Valores faltantes
* Posibles duplicaciones
* Campos con mezcla de campos de tipo numérico con cadenas de caracteres

En esta clase vamos a detectar algunos de los qué campos presentan problemas, y solucionarlos usando las herramientas que adquirimos en las prácticas guiadas.


## Ejercicio 1:

Vamos a leer en la variable `data` los datos del archivo /M2/CLASE_05_Limpieza_de_datos/Data/MetObjects_sample.csv en un `DataFrame` de pandas con el método `read_csv` 

Veamos cuántas filas y columnas tiene el DataFrame data

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('../Data/MetObjects_sample.csv', low_memory=False)

In [3]:
df.tail()

,Unnamed: 0,Object Number,Is Highlight,Is Public Domain,Is Timeline Work,Object ID,Department,AccessionYear,Object Name,Title,...,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL
4738,55188,CP288,False,False,False,63654,Asian Art,1941,Print,NaN,...,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN
4739,333892,1979.292.676,False,False,False,496985,Modern and Contemporary Art,1979,Drawing,Untitled (female nude studies),...,NaN,NaN,Drawings,"© 2020 Artists Rights Society (ARS), New York",http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN
4740,189623,1979.206.698,False,False,True,312885,"Arts of Africa, Oceania, and the Americas",1979,Bowl,Bowl,...,NaN,NaN,Ceramics-Containers,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN
4741,213869,53.605.2,False,False,False,345597,Drawings and Prints,1953,Book,Anatomia ex Caspari Bartolini Parentis Institu...,...,NaN,NaN,Books,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN
4742,473188,"Burdick 335, V31.4",False,False,False,841114,Drawings and Prints,1963,Print,"Roy ""Peewee"" Chantler from Dominion Chocolate ...",...,NaN,NaN,Prints|Ephemera,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN


## Ejercicio 2: Formato y tipos de datos

Vamos a ver de qué tipo de datos es cada columna del DataFrame, y vamos a convertir o dar formato **a alguna de las columnas** que tienen tipo de datos incorrecto.

### 2.a Detectar las columnas que tienen tipo de datos incorrecto

In [4]:
df.dtypes

Unnamed: 0                   int64
Object Number               object
Is Highlight                  bool
Is Public Domain              bool
Is Timeline Work              bool
Object ID                    int64
Department                  object
AccessionYear               object
Object Name                 object
Title                       object
Culture                     object
Period                      object
Dynasty                     object
Reign                       object
Portfolio                   object
Artist Role                 object
Artist Prefix               object
Artist Display Name         object
Artist Display Bio          object
Artist Suffix               object
Artist Alpha Sort           object
Artist Nationality          object
Artist Begin Date           object
Artist End Date             object
Artist Gender               object
Artist ULAN URL             object
Artist Wikidata URL         object
Object Date                 object
Object Begin Date   

### 2.b AccessionYear

Analizar la columna AccessionYear que fue leída como object, y debería ser int.

¿Qué valores toma ese campo? ¿Cómo se distribuyen esos valores? ¿Hay valores nulos?

Queremos extraer el dato año de los valores no numéricos, y crear una nueva columna en el DataFrame de tipo int que se llame AccessionYearClean y tenga estos valores.

Para eso vamos a usar expresiones regulares, apply y lambda.

Observación: si la columna tiene valores NaN no vamos a poder convertirla al tipo int. Una opción es reemplazar los valores NaN por algún entero antes de convertir (`fillna`). La otra opción es dejar la columna como tipo float (el tipo de NaN es float).


In [5]:
df['AccessionYear'].unique()

array([nan, '2005-02-15', '2020-03-23', '1986-12-29', '1992', '1991',
       '1975', '1930', '1874', '1915', '1920', '1921', '1916', '1929',
       '1907', '1989', '1962', '1917', '1895', '1950', '1896', '1924',
       '1951', '1970', '1958', '1938', '2006', '1953', '1891', '1965',
       '1966', '2009', '1911', '1947', '1925', '1955', '1914', '1928',
       '2012', '1926', '2015', '1879', '1887', '1949', '1902', '1981',
       '1979', '1899', '1918', '1906', '1912', '2001', '1985', '1976',
       '1946', '1927', '1973', '1941', '1900', '2019', '2013', '1942',
       '1968', '1967', '1988', '1986', '1933', '1913', '1909', '1977',
       '1922', '1964', '2018', '1936', '1948', '1919', '1954', '1959',
       '1931', '1943', '1908', '1945', '1932', '1982', '1939', '1881',
       '1956', '2002', '1910', '2008', '1923', '1961', '1937', '2004',
       '1971', '1905', '1893', '1898', '1994', '1960', '1957', '1880',
       '1984', '2016', '1944', '1969', '1972', '1871', '1987', '2007',
       

In [6]:
df['AccessionYear'].isna().sum(), df['AccessionYear'].shape

(1000, (4743,))

In [7]:
pattern="(?P<year>\d{4})"
pattern_c = re.compile(pattern)
result = df['AccessionYear'].apply(lambda x: x if x is np.NaN else re.match(pattern_c,x))

print('Valores nulos luego de extraer el año: ', result.isna().sum()) # Si da 1000, confirma que tomé todos los valores

df['AccessionYearClean'] = result\
                                .apply(lambda x: x if x is np.NaN else x.group())\
                                .fillna('-1')\
                                .astype('int')

Valores nulos luego de extraer el año:  1000


In [8]:
df['AccessionYearClean'].unique()

array([  -1, 2005, 2020, 1986, 1992, 1991, 1975, 1930, 1874, 1915, 1920,
       1921, 1916, 1929, 1907, 1989, 1962, 1917, 1895, 1950, 1896, 1924,
       1951, 1970, 1958, 1938, 2006, 1953, 1891, 1965, 1966, 2009, 1911,
       1947, 1925, 1955, 1914, 1928, 2012, 1926, 2015, 1879, 1887, 1949,
       1902, 1981, 1979, 1899, 1918, 1906, 1912, 2001, 1985, 1976, 1946,
       1927, 1973, 1941, 1900, 2019, 2013, 1942, 1968, 1967, 1988, 1933,
       1913, 1909, 1977, 1922, 1964, 2018, 1936, 1948, 1919, 1954, 1959,
       1931, 1943, 1908, 1945, 1932, 1982, 1939, 1881, 1956, 2002, 1910,
       2008, 1923, 1961, 1937, 2004, 1971, 1905, 1893, 1898, 1994, 1960,
       1957, 1880, 1984, 2016, 1944, 1969, 1972, 1871, 1987, 2007, 1980,
       2000, 1990, 1892, 1995, 1974, 1978, 1940, 1983, 1952, 2011, 1889,
       1993, 1996, 1886, 1963, 2003, 2017, 1997, 1901, 2014, 1883, 2010,
       1935, 1999, 1894, 1934, 1998, 1904, 1897, 1890, 1873])

## Ejercicio 3: Categorias - Valores erróneos

Miremos ahora el campo "Artist Gender"

¿Qué valores toma ese campo? ¿Cómo se distribuyen esos valores? ¿Hay valores nulos?

Queremos definir como categorías válidas Male, Female y Unknown

Y crear una nueva columna en el DataFrame que se llame ArtistGenderClean y tenga estos valores.

Para eso vamos a usar expresiones regulares, apply y lambda.

¿Podemos deducir cómo está representada la categoría Male en el dataset original?

Nota: La propuesta que hacemos para limpiar este campo no es del todo correcta, y vamos a ver por qué más adelante. Pero sirve como ejercicio.


In [14]:
print(df['Artist Gender'].isna().sum())
df['ArtistGenderClean'] = df['Artist Gender'].fillna('Unknown')
df['ArtistGenderClean'].value_counts()

3751


Unknown                     3751
|                            583
||                           170
Female                        81
Female|                       43
|||                           41
|Female                       23
||||                          20
||||||                         7
|||||||                        3
Female||                       3
|||||                          3
Female||Female                 2
|Female|                       2
|||||||||||                    2
||||||||                       1
Female|Female                  1
|Female|Female|||Female        1
|||||||||||||||||              1
|Female||                      1
|||Female|||||                 1
||Female                       1
||Female|                      1
|||||||||||||||Female|||       1
Name: ArtistGenderClean, dtype: int64

In [10]:
df['Artist Gender'].value_counts()

|                           583
||                          170
Female                       81
Female|                      43
|||                          41
|Female                      23
||||                         20
||||||                        7
|||||||                       3
|||||                         3
Female||                      3
Female||Female                2
|Female|                      2
|||||||||||                   2
|Female|Female|||Female       1
Female|Female                 1
||||||||                      1
|Female||                     1
|||||||||||||||||             1
|||Female|||||                1
||Female                      1
||Female|                     1
|||||||||||||||Female|||      1
Name: Artist Gender, dtype: int64

In [25]:
pattern_pipe = '\|+'
regex_pipe = re.compile(pattern)
df['ArtistGenderClean'] = df['ArtistGenderClean'].apply(lambda x: regex_pipe.sub('',x))

In [39]:
pattern_female = '(Female)+'
regex_female = re.compile(pattern_female)
df['ArtistGenderClean'] = df['ArtistGenderClean'].apply(lambda x: regex_female.sub('Female',x))

In [43]:
mask_empty = df['ArtistGenderClean'] == ''
df.loc[mask_empty,'ArtistGenderClean'] = 'Male'

In [44]:
df['ArtistGenderClean'].value_counts()

Unknown    3751
Male        831
Female      161
Name: ArtistGenderClean, dtype: int64

## Ejercicio 4: Imputación

Vamos a analizar ahora los campos "Object Date", "Object Begin Date", "Object End Date"

**4.a ¿Cuántos valores nulos hay en "Object Date"? ¿Cuántos en "Object Begin Date"? ¿Cuántos en "Object End Date"?**


In [49]:
print('valores nulos en Object Date: ',df['Object Date'].isna().sum())
print('valores nulos en Object Begin Date: ',df['Object Begin Date'].isna().sum())
print('valores nulos en Object End Date: ',df['Object End Date'].isna().sum())

valores nulos en Object Date:  1570
valores nulos en Object Begin Date:  0
valores nulos en Object End Date:  0


**4.b Usaremos los valores de "Object Begin Date" o "Object End Date" para imputar los valores de "Object Date" con alguno de esos dos campos.**

1) Vamos a crear una columna nueva ("Object Date 4b") donde copiamos todos los datos de Object Date (para no cambiar los valores originales y nos sirvan para el próximo ejercicio)

2) Vamos a rellenar la columna "Object Date 4b" con la estrategia que planteamos.

In [51]:
# 1)
df['Object Date 4b'] = df['Object Date'].copy()

In [65]:
df.iloc[4720:4742].loc[:,['Object Date','Object Begin Date', 'Object End Date']]

,Object Date,Object Begin Date,Object End Date
4720,April 1861,1861,1861
4721,ca. 1925–27,1925,1927
4722,1888,1888,1888
4723,ca. 1550,1525,1575
4724,1880s,1880,1889
4725,1940s,1940,1949
4726,late 6th/early 5th centuries B.C.,-700,-300
4727,1763,1763,1763
4728,first half 18th century,1700,1750
4729,1934,1934,1934


No hay un criterio unificado para imputar la columna Object Date, ya que no sigue un patrón coherente. Usa indistintamente el inicio, fin, promedio, rango, siglo, período, etc

**4.c Usaremos aleatoriamente los valores de "Object Begin Date" o "Object End Date" para imputar los valores de "Object Date" con alguno de esos dos campos.**

Para eso definimos una función get_fill_value que recibe como parámetro una fila da data, y si el valor del campo "Object Date" es nulo devuelve aleatoriamente el valor del campo "Object Begin Date" u "Object End Date" de ese registro.

(En este ejercicio practicamos todo!)

In [181]:
def get_fill_value(row):
    feature = ['Object Begin Date', 'Object End Date']
    if row['Object Date'] is np.nan:
        result = row[feature[np.random.randint(2)]]
    else:
        result = row['Object Date']
    return result

In [203]:
df['Object Date 4b'] = df.apply(get_fill_value, axis=1)
df['Object Date 4b'].isna().sum()

0

Vamos a contar ahora cuántos registros rellenó con los valores de Object Begin Date y cuántos con Object End Date

In [224]:
mask_filled = df['Object Date'] != df['Object Date 4b']
mask_begin = df[mask_filled]['Object Begin Date'] == df[mask_filled]['Object Date 4b']
print(mask_filled.sum())
print('Rellenado con el valor de Begin:')
mask_begin.value_counts()

1570
Rellenado con el valor de Begin:


True     850
False    720
dtype: int64

In [221]:
# Alternativa para reducir el error cuando son iguales los datos. cuento los de end también.
mask_end = df[mask_filled]['Object End Date'] == df[mask_filled]['Object Date 4b']
print('Con Begin (contando los que son iguales): ', mask_begin.sum())
print('Con End (contando los que son iguales): ', mask_end.sum())

Con Begin (contando los que son iguales):  850
Con End (contando los que son iguales):  877


## Nota: Organizar el dataset

Para llevar a cabo esta tarea necesitamos algunas herramientas que veremos en la segunda parte de Pandas.

En esa clase vamos a volver a este ejercicio y resolver este punto.

Los campos que vamos a analizar son "Artist Nationality" y "Artist Display Name"

In [222]:
df["Artist Nationality"].value_counts()

American                                                                                     329
Austrian                                                                                     153
French                                                                                       146
Japanese                                                                                      96
Italian                                                                                       93
                                                                                            ... 
German|German|British                                                                          1
American|American|American|American, born Germany|American, born Canada|American|American      1
French|Japanese                                                                                1
French, born Russia|French                                                                     1
Italian|German|British        

In [223]:
df["Artist Display Name"].value_counts() 

Walker Evans                                                             35
Unknown                                                                  32
W. Duke, Sons & Co.                                                      21
Kinney Brothers Tobacco Company                                          20
Goodwin & Company                                                        19
                                                                         ..
Yohji Yamamoto                                                            1
Maxime Du Camp                                                            1
Cheney Brothers|Associated Artists                                        1
Bernhardt Christoph Anckerman|Johann Georg Ringlin|Martin Engelbrecht     1
Franz Lebisch|Wiener Werkstätte                                           1
Name: Artist Display Name, Length: 1595, dtype: int64